In [14]:
import pandas as pd
import os

In [30]:
sample_sheet = pd.read_csv("~/Desktop/gdc_sample_sheet.2025-05-20.tsv", sep = "\t")
sample_sheet['patient_id'] = sample_sheet['Case ID'].str.split(',', n=1).str[0]
sample_sheet

,File ID,File Name,Data Category,Data Type,Project ID,Case ID,Sample ID,Tissue Type,Tumor Descriptor,Specimen Type,Preservation Method,patient_id
0,4d8f5377-1b3f-42fc-b30d-56676e6fcff1,TCGA-HNSC.b24695ff-10a8-480f-885a-3da3356b3678...,Copy Number Variation,Gene Level Copy Number,TCGA-HNSC,TCGA-CN-6024,TCGA-CN-6024-01A,Tumor,Primary,Solid Tissue,Unknown,TCGA-CN-6024
1,0dedce03-c368-4db8-837d-3d1cbfc07b44,TCGA-HNSC.a3a3e8ac-0a4d-471e-abaf-869184483285...,Copy Number Variation,Gene Level Copy Number,TCGA-HNSC,TCGA-CQ-7068,TCGA-CQ-7068-01A,Tumor,Primary,Solid Tissue,Unknown,TCGA-CQ-7068
2,2b382e9d-3727-48bc-adad-7942c04e9ed7,TCGA-HNSC.f8be7bc9-dbbd-4cc7-93eb-066f3aa522db...,Copy Number Variation,Gene Level Copy Number,TCGA-HNSC,TCGA-HD-7229,TCGA-HD-7229-01A,Tumor,Primary,Solid Tissue,Unknown,TCGA-HD-7229
3,e7952a90-ef15-43da-97f7-14d3bbfefcba,TCGA-HNSC.dc985e57-caef-4c8f-adc9-92d035701c3e...,Copy Number Variation,Gene Level Copy Number,TCGA-HNSC,TCGA-BB-4227,TCGA-BB-4227-01A,Tumor,Primary,Solid Tissue,Unknown,TCGA-BB-4227
4,5a33ddaa-89bc-4062-9a52-be54702306d7,TCGA-HNSC.5d07e7cd-558a-41ac-b066-b4f234522a96...,Copy Number Variation,Gene Level Copy Number,TCGA-HNSC,TCGA-CQ-A4C7,TCGA-CQ-A4C7-01A,Tumor,Primary,Solid Tissue,Unknown,TCGA-CQ-A4C7
...,...,...,...,...,...,...,...,...,...,...,...,...
510,6f31f6c3-7f63-48fc-bcb7-67168e2e76e5,TCGA-HNSC.a3e9465b-c53b-4aa4-b2ac-fe3fa9a76c47...,Copy Number Variation,Gene Level Copy Number,TCGA-HNSC,TCGA-CX-7082,TCGA-CX-7082-01A,Tumor,Primary,Solid Tissue,Unknown,TCGA-CX-7082
511,eb8feb46-7023-425b-87f8-15bf86f48502,TCGA-HNSC.6ecd61db-f545-4831-a67c-dd6f3914b308...,Copy Number Variation,Gene Level Copy Number,TCGA-HNSC,TCGA-CV-7103,TCGA-CV-7103-01A,Tumor,Primary,Solid Tissue,Unknown,TCGA-CV-7103
512,ba14e621-740b-4997-8d63-f6bce229e9bc,TCGA-HNSC.73309f6f-8b62-43a9-bae1-b2ba8fc98c64...,Copy Number Variation,Gene Level Copy Number,TCGA-HNSC,TCGA-BA-A6DG,TCGA-BA-A6DG-01A,Tumor,Primary,Unknown,Unknown,TCGA-BA-A6DG
513,f07fbde1-362d-43a9-8203-68d3eb62c449,TCGA-HNSC.a42cb29c-9cc4-4d5c-a525-56f1d1c657ae...,Copy Number Variation,Gene Level Copy Number,TCGA-HNSC,TCGA-H7-A6C5,TCGA-H7-A6C5-01A,Tumor,Primary,Unknown,Unknown,TCGA-H7-A6C5


In [50]:
sample_info = pd.read_csv("~/Desktop/TCGA_mastercalls.abs_tables_JSedit.fixed.txt", sep = "\t")
sample_info['patient_id'] = (
    sample_info['array']
      .str.split('-')
      .str[:3]
      .str.join('-')
)
sample_info = sample_info[sample_info['patient_id'].isin(sample_sheet['patient_id'])]
sample_info[sample_info["Genome doublings"] == 2]

,array,sample,call status,purity,ploidy,Genome doublings,Coverage for 80% power,Cancer DNA fraction,Subclonal genome fraction,solution,patient_id
2842,TCGA-BA-5151-01,TCGA-BA-5151-01A-01D-1432-01,called,0.62,4.11,2.0,25.0,0.77,0.41,new,TCGA-BA-5151
2936,TCGA-CN-6995-01,TCGA-CN-6995-01A-31D-2010-01,called,0.37,3.70,2.0,38.0,0.52,0.25,new,TCGA-CN-6995
2939,TCGA-CN-6998-01,TCGA-CN-6998-01A-23D-2010-01,called,0.46,4.85,2.0,38.0,0.67,0.08,new,TCGA-CN-6998
2986,TCGA-CQ-A4C9-01,TCGA-CQ-A4C9-01A-11D-A253-01,called,0.41,4.28,2.0,37.0,0.60,0.40,new,TCGA-CQ-A4C9
3003,TCGA-CR-6474-01,TCGA-CR-6474-01A-11D-1869-01,called,0.20,4.01,2.0,66.0,0.33,0.26,new,TCGA-CR-6474
3045,TCGA-CV-5432-01,TCGA-CV-5432-01A-02D-1682-01,called,0.47,5.35,2.0,40.0,0.70,0.12,new,TCGA-CV-5432
3047,TCGA-CV-5435-01,TCGA-CV-5435-01A-01D-1682-01,called,0.66,3.71,2.0,20.0,0.78,0.15,new,TCGA-CV-5435
3080,TCGA-CV-6950-01,TCGA-CV-6950-01A-11D-1910-01,called,0.44,4.25,2.0,35.0,0.62,0.08,new,TCGA-CV-6950
3112,TCGA-CV-7247-01,TCGA-CV-7247-01A-11D-2010-01,called,0.46,4.96,2.0,39.0,0.68,0.39,new,TCGA-CV-7247
3132,TCGA-CV-7423-01,TCGA-CV-7423-01A-11D-2077-01,called,0.47,4.27,2.0,33.0,0.66,0.35,new,TCGA-CV-7423


In [55]:
base_dir = os.path.expanduser("~/Desktop/cnv/")

import pandas as pd
import os

status_dict = {}

for patient in sample_sheet["patient_id"].unique():
    fn = (sample_sheet
        .loc[sample_sheet["patient_id"] == patient, "File Name"]
        .astype(str)
        .iloc[0]
    )
    df = pd.read_csv(os.path.join(base_dir, fn), sep="\t")
    df = df.dropna(subset=['copy_number'])
    df = df.drop_duplicates(subset='gene_name', keep='first')

    ploidy = sample_info.loc[sample_info["patient_id"] == patient, "ploidy"].iloc[0]

    df['cnv_status'] = df['copy_number'].apply(
        lambda x: 1 if x >= ploidy + 0.5
                  else (-1 if x <= ploidy - 0.5
                        else 0)
    )

    status_dict[patient] = df.set_index('gene_name')['cnv_status']

cnv_status_df = pd.DataFrame(status_dict)

cnv_status_df


,TCGA-CN-6024,TCGA-CQ-7068,TCGA-HD-7229,TCGA-BB-4227,TCGA-CQ-A4C7,TCGA-BA-6871,TCGA-BB-4223,TCGA-CX-7219,TCGA-CV-7410,TCGA-BA-5557,...,TCGA-D6-A6EQ,TCGA-P3-A5Q6,TCGA-CR-6484,TCGA-BA-A6DA,TCGA-BA-4075,TCGA-CX-7082,TCGA-CV-7103,TCGA-BA-A6DG,TCGA-H7-A6C5,TCGA-CR-7393
gene_name,,,,,,,,,,,,,,,,,,,,,
5S_rRNA,0.0,0.0,0.0,1.0,-1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0
5_8S_rRNA,1.0,0.0,0.0,0.0,1.0,-1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,NaN,0.0,0.0,1.0,0.0,0.0
7SK,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,...,0.0,-1.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0
A1BG,0.0,0.0,0.0,0.0,-1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0
A1BG-AS1,0.0,0.0,0.0,0.0,-1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZZEF1,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,-1.0,0.0,-1.0,...,-1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0
ZZZ3,-1.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,...,0.0,-1.0,0.0,0.0,-1.0,0.0,-1.0,1.0,0.0,0.0
hsa-mir-1253,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,-1.0,0.0,-1.0,...,-1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0


In [68]:
na_frac = cnv_status_df.isna().mean(axis=1)
filtered_cnv_status = cnv_status_df[na_frac <= 0.2]

In [71]:
meta_data = pd.read_csv("~/Desktop/BNFO285_Projects/Data/processed/processed_metadata.txt", sep="\t")
patients_of_interest = meta_data["patient_id"]
filtered_cnv_status = filtered_cnv_status[list(set(filtered_cnv_status.columns).intersection(patients_of_interest))]
filtered_cnv_status

,TCGA-CR-6473,TCGA-HD-7754,TCGA-CN-5360,TCGA-WA-A7H4,TCGA-CV-A6JU,TCGA-CN-6011,TCGA-CR-7394,TCGA-CN-4733,TCGA-T3-A92N,TCGA-CQ-7063,...,TCGA-CN-6024,TCGA-CN-A63V,TCGA-CN-6988,TCGA-CR-6478,TCGA-CR-6488,TCGA-CR-7382,TCGA-BA-5555,TCGA-DQ-7588,TCGA-CQ-5325,TCGA-P3-A6SX
gene_name,,,,,,,,,,,,,,,,,,,,,
5S_rRNA,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
5_8S_rRNA,1.0,0.0,-1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,NaN,1.0,0.0,0.0,1.0,0.0,0.0
7SK,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,...,0.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,1.0
A1BG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0
A1BG-AS1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZZEF1,0.0,0.0,-1.0,1.0,-1.0,0.0,0.0,0.0,-1.0,0.0,...,0.0,0.0,-1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
ZZZ3,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
hsa-mir-1253,0.0,0.0,-1.0,1.0,-1.0,0.0,0.0,0.0,-1.0,0.0,...,0.0,0.0,-1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [81]:
gtf = (
    pd.read_csv(
        "~/Desktop/gencode.v38.annotation.gtf",
        sep="\t",
        comment="#",
        header=None,
        names=[
            "chrom","source","feature",
            "start","end","score","strand","frame","attribute"
        ],
    )
    .query("feature == 'gene'")
)

def _parse_attr(s):
    d = {}
    for field in s.split(";"):
        if not field.strip(): 
            continue
        key, val = field.strip().split(" ", 1)
        d[key] = val.strip('"')
    return d

attrs = gtf["attribute"].apply(_parse_attr)
gtf["gene_name"] = attrs.map(lambda d: d.get("gene_name"))
gtf["gene_type"] = attrs.map(lambda d: d.get("gene_type"))

coding = set(
    gtf
    .loc[gtf["gene_type"] == "protein_coding", "gene_name"]
    .dropna()
)

In [82]:
filtered_coding = filtered_cnv_status.loc[filtered_cnv_status.index.isin(coding)]
filtered_coding

,TCGA-CR-6473,TCGA-HD-7754,TCGA-CN-5360,TCGA-WA-A7H4,TCGA-CV-A6JU,TCGA-CN-6011,TCGA-CR-7394,TCGA-CN-4733,TCGA-T3-A92N,TCGA-CQ-7063,...,TCGA-CN-6024,TCGA-CN-A63V,TCGA-CN-6988,TCGA-CR-6478,TCGA-CR-6488,TCGA-CR-7382,TCGA-BA-5555,TCGA-DQ-7588,TCGA-CQ-5325,TCGA-P3-A6SX
gene_name,,,,,,,,,,,,,,,,,,,,,
A1BG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0
A1CF,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,-1.0,-1.0,0.0,0.0,0.0
A2M,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
A2ML1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
A3GALT2,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11A,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
ZYG11B,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
ZYX,0.0,0.0,1.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,-1.0,-1.0,0.0,0.0,0.0,-1.0,0.0,-1.0


In [83]:
filtered_coding.to_csv("~/Desktop/BNFO285_Projects/Data/processed/filtered_cnv_status.tsv", index=True, sep = "\t")